In [1]:

import string
import pandas as pd 
from tqdm.autonotebook import tqdm

from utils.RagPipeline import Ragpipeline

from utils.utils import normalize_path

C:\Users\kim_h\AppData\Local\Temp\ipykernel_8080\3463511841.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("../open/sample_submission.csv")

df = pd.read_csv('../open/test.csv')
base_directory = '../open' # Your Base Directory

df.head()

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?


In [3]:
source_list = list(df['Source'].unique())
# 알파벳 리스트
alphabet = list(string.ascii_uppercase)

# 딕셔너리로 파일명과 알파벳 매핑
file_mapping = {alphabet[i]: source_list[i] for i in range(len(source_list))}

reversed_file_mapping = {value: key for key, value in file_mapping.items()}

file_mapping

{'A': '중소벤처기업부_혁신창업사업화자금(융자)',
 'B': '보건복지부_부모급여(영아수당) 지원',
 'C': '보건복지부_노인장기요양보험 사업운영',
 'D': '산업통상자원부_에너지바우처',
 'E': '국토교통부_행복주택출자',
 'F': '「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》',
 'G': '「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》',
 'H': '「FIS 이슈&포커스」 22-2호 《재정성과관리제도》',
 'I': '「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》'}

In [4]:
# 대략 40분 걸림 
from config import config 

for i, source in enumerate(source_list):
    
    db_path = f'data/{reversed_file_mapping[source]}'
    pipeline = Ragpipeline(db_path, config)
    
    target_df = df[df['Source'] == source]
    
    for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions ({i+1}/{len(source_list)})"):
        # 소스 문자열 정규화
        source = normalize_path(row['Source'])
        # 질문 
        question = row['Question']
        
        # 대답
        full_response = pipeline.answer_generation(question)
        first_line_response = full_response.split("\n")[0]
        
        # 결과 저장
        submit_df.loc[submit_df['SAMPLE_ID'] == row['SAMPLE_ID'], 'Answer'] = first_line_response
        

Answering Questions (9/9): 100%|██████████| 20/20 [42:38<00:00, 127.90s/it]


In [5]:
submit_df

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자의) 예산은 2억 3,000만 원입니다."
1,TEST_001,중소벤처기업부의 혁신창업사업화자금(융자)의 사업목적은 기술력과 사업성이 우수하고 미...
2,TEST_002,중소벤처기업부의 혁신창업사업화자금(융자)의 사업근거는 중소기업진흥에 관한 법률 제6...
3,TEST_003,2010년 3월에 재창업자금(실패 경영인에 대한 재기지원)을 신규로 지원하였습니다.
4,TEST_004,"혁신창업사업화자금 중 2020년에 신규 지원된 자금으로 미래기술육성자금, 고성장촉진..."
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,미래사회보장급여에 대한 순의무는 사회보장제도의 미래보험료의 순현재가치를 차감한 것을...
95,TEST_095,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리는 다릅니...
96,TEST_096,"우발부채는 미래 발생이 불확실한 잠재적 의무 또는 부채를 의미하며, 이는 과거의 거..."


In [ ]:
import os 
save_path = './results'
os.makedirs(save_path, exist_ok=True)

In [6]:
submit_df.to_csv(f"./{save_path}/Faiss_lamma31_ensemble_submission2.csv", encoding='UTF-8-sig', index=False)